## The Two Body Problem Numerically

We will solve the two body problem <br>
$$\ddot{\bf{r}} = \frac{GM}{r^2}\hat{\bf{r}}$$
Where $\bf{r}$ is the relative position of one body (called planet here) with respect to another body (called star here). M is the total mass of the system, and G is the gravitational constant (normalised to 1 in this program)

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# gala has Leapfrog implemented in its integrator. Try using that instead of the slow Python for loop.

In [ ]:
G = 1

In [ ]:
class pointParticle:
    """
        Class to implement a point particle in 2D in a gravitationally interacting system. 
        Attributes are
        Mass, Position and Velocity. Also derived are Linear Momentum and Kinetic Energy.
        Given another body, we can calculate the force, acceleration and Potential Energy. 
    """
    def __init__(self, mass, position, velocity):
        self._mass = mass
        self._x, self._y = position
        self._vx, self._vy = velocity
        
    @property
    def mass(self):
        return self._mass
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    @property
    def vx(self):
        return self._vx
    
    @property
    def vy(self):
        return self._vy
    
    @property
    def position(self):
        return np.array([self._x, self._y])
    
    @property
    def velocity(self):
        return np.array([self._vx, self._vy])
        
    @property
    def momentum(self):
        return self.mass*self.velocity
    
    def distance(self, mass2):
        return np.sqrt((self._x-mass2.x)**2 + (self._y-mass2.y)**2)
    
    def rel_position(self, mass2):
        return np.array([mass2.x - self._x, mass2.y - self._y])
    
    def rel_velocity(self, mass2):
        return np.array([mass2.vx - self._vx, mass2.vy - self._vy])
    
    def grav_force(self, mass2):
        return G*self.mass*mass2.mass/(self.distance(mass2))**3*self.rel_position(mass2)
    
    def acceleration(self, mass2):
        return self.grav_force(mass2)/self.mass
    
    def update_pos(self, position):
        self._x, self._y = position
        
    def update_vel(self, velocity):
        self._vx, self._vy = velocity
        
    def potential_energy(self, mass2):
        return -G*self.mass*mass2.mass/self.distance(mass2)
    
    def kinetic_energy(self):
        return 1/2*self.mass*np.linalg.norm(self.velocity)**2
    
    # Old relic, useful if you use scipy's standard ODE solvers
#     @property
#     def fourpos(self):
#         return np.r_[self.position, self.velocity]
    
#     def fourvel(self, mass2):
#         return np.r_[self.velocity, self.acceleration(mass2)]

In [ ]:
# Define the time-resolution of the integration.
dt = 0.0001

# Time range of the integration
time_range = np.arange(0,10,dt)

In [ ]:
planet = pointParticle(0.0001, [1,0], [0,0.5])

In [ ]:
star = pointParticle(1, [0,0], [0,0])

In [ ]:
# # Euler Integration
# x = [planet.position]
# v = [planet.velocity]
# E = [planet.kinetic_energy()+planet.potential_energy(star)]
# for i in range(time_range.size):
#     x_, v_ = x[-1]+v[-1]*dt, v[-1] + planet.acceleration(star)*dt
#     x.append(x_)
#     v.append(v_)
#     planet.update_pos(x[-1])
#     planet.update_vel(v[-1])
#     E.append(planet.kinetic_energy()+planet.potential_energy(star))
# pos = np.array(x)
# vel = np.array(v)
# energy = np.array(E)

In [ ]:
x_star = [star.position]
star.update_vel(star.velocity+star.acceleration(planet)*dt/2)
v_star = [star.velocity]

x_planet = [planet.position]
planet.update_vel(planet.velocity+planet.acceleration(star)*dt/2)
v_planet = [planet.velocity]

linear_momentum = [planet.momentum+star.momentum]

In [ ]:
for i in range(time_range.size):
    v_planet.append(v_planet[-1] + planet.acceleration(star)*dt)
    v_star.append(v_star[-1] + star.acceleration(planet)*dt)    
    planet.update_vel(v_planet[-1])
    star.update_vel(v_star[-1])
    x_planet.append(x_planet[-1] + v_planet[-1]*dt)
    x_star.append(x_star[-1] + v_star[-1]*dt)
    planet.update_pos(x_planet[-1])
    star.update_pos(x_star[-1])
    linear_momentum.append(planet.momentum+star.momentum)
    
pos_planet = np.array(x_planet)
vel_planet = np.array(v_planet)

pos_star = np.array(x_star)
vel_star = np.array(v_star)

linear_momentum = np.linalg.norm(np.array(linear_momentum), axis=1)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(pos_planet[:,0], pos_planet[:,1], s=1, alpha=0.4)
plt.scatter(pos_star[:,0], pos_star[:,1], s=20, alpha=0.4)

In [ ]:
a = (pos_planet[:,0].max()-pos_planet[:,0].min())/2
ae =(pos_planet[:,0].max()+pos_planet[:,0].min())/2 - pos_star[:,0].mean()
e = ae/a
b = (pos_planet[:,1].max() - pos_planet[:,1].min())/2

In [ ]:
print('Semi-major axis = {:2.4f}\nEccentricity = {:2.4f}'.format(a, e))

In [ ]:
print("Theoretical Time Period = {:2.5f}".format(2*np.pi*np.sqrt(a**3/(G*(planet.mass+star.mass)))))

In [ ]:
energy_system = -G*planet.mass*star.mass/np.linalg.norm(pos_planet-pos_star, axis=1)+\
                0.5*planet.mass*np.linalg.norm(vel_planet, axis=1)**2 + \
                0.5*star.mass*np.linalg.norm(vel_star, axis=1)

In [ ]:
# plt.plot(time_range, energy_planet[:-1])
# plt.plot(time_range, energy_star[:-1])
# plt.figure(figsize=(8,6))
plt.plot(time_range, energy_system[:-1]);
# plt.suptitle('Energy = {} +- {}\nTimestep = {}% of orbital period'.format(energy_system.mean(), energy_system.std(), dt*2**0.5/np.pi*100))
# plt.savefig('TestingLeapfrog.png')

In [ ]:
np.where(np.abs(energy_system-energy_system[0])<0.000000002)

In [ ]:
plt.plot((pos_planet-pos_star))

Changing the initial conditions and the masses can result in many more possible orbits